Processing Data

In [43]:
import pandas as pd
import numpy as np
import sqlite3

connection = sqlite3.connect("focus.db")
df = pd.read_sql("SELECT * FROM focus_logs",connection)
#The first thing is to add a tracking ended entry. So...
df

,id,date,time,focused,program,session_end
0,1,09/07/2024,13:54:43,focus.py - TimeLens - Visual Studio Code,Code.exe,0
1,2,09/07/2024,13:54:45,Efficient Time Calculation - Google Chrome,chrome.exe,0
2,3,09/07/2024,13:54:49,"Soft Jazz Music for Study, Work, Focus☕Relaxin...",chrome.exe,0
3,4,09/07/2024,13:55:09,Efficient Time Calculation - Google Chrome,chrome.exe,0
4,5,09/07/2024,13:55:10,"Soft Jazz Music for Study, Work, Focus☕Relaxin...",chrome.exe,0
...,...,...,...,...,...,...
116,117,09/07/2024,16:27:35,#vaping-corner | The Jade Palace - Discord,Discord.exe,0
117,118,09/07/2024,16:27:37,VALORANT,VALORANT-Win64-Shipping.exe,0
118,119,09/07/2024,16:37:20,#vaping-corner | The Jade Palace - Discord,Discord.exe,0
119,120,09/07/2024,16:37:22,● processing.ipynb - TimeLens - Visual Studio ...,Code.exe,0


In [44]:
programInsights = pd.read_sql("SELECT * FROM program_insights",connection)
#The first thing is to add a tracking ended entry. So...
programInsights


,id,program,total_time,last_updated
0,1,Code.exe,0 days 00:06:26,2024-09-07 20:37:23
1,2,None,0 days 01:29:22,2024-09-07 20:37:23
2,3,chrome.exe,0 days 00:11:31,2024-09-07 20:37:23
3,8,Discord.exe,0 days 00:01:22,2024-09-07 20:37:23
4,14,SearchApp.exe,0 days 00:00:02,2024-09-07 20:37:23
5,16,notepad.exe,0 days 00:00:27,2024-09-07 20:37:23
6,21,Spotify.exe,0 days 00:01:01,2024-09-07 20:37:23
7,22,VALORANT-Win64-Shipping.exe,0 days 00:52:30,2024-09-07 20:37:23
8,24,explorer.exe,0 days 00:00:04,2024-09-07 20:37:23


In [45]:
generalInsights = pd.read_sql("SELECT * FROM general_insights",connection)
#The first thing is to add a tracking ended entry. So...
generalInsights

,id,total_context_switches,last_updated


In [25]:
df = pd.read_sql("SELECT * FROM focus_logs",connection)
#The first thing is to add a tracking ended entry. So...
# Now by entry, I want to get a total time/add up datetimes. Join "Date" and "Time" then just sum up?
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
df = df.sort_values(by=['datetime'])

# Current datetime - next datetime
df['timespent'] = df['datetime'].shift(-1) - df['datetime']
# For the last row, set a default time (e.g., 5 seconds), as there's no next row
df.iloc[-1, df.columns.get_loc('timespent')] = pd.Timedelta(seconds=5)

total_time_per_program = df.groupby('program')['timespent'].sum()
df

,id,date,time,focused,program,session_end,datetime,timespent
0,1,09/07/2024,13:54:43,focus.py - TimeLens - Visual Studio Code,Code.exe,0,2024-09-07 13:54:43,0 days 00:00:02
1,2,09/07/2024,13:54:45,Efficient Time Calculation - Google Chrome,chrome.exe,0,2024-09-07 13:54:45,0 days 00:00:04
2,3,09/07/2024,13:54:49,"Soft Jazz Music for Study, Work, Focus☕Relaxin...",chrome.exe,0,2024-09-07 13:54:49,0 days 00:00:20
3,4,09/07/2024,13:55:09,Efficient Time Calculation - Google Chrome,chrome.exe,0,2024-09-07 13:55:09,0 days 00:00:01
4,5,09/07/2024,13:55:10,"Soft Jazz Music for Study, Work, Focus☕Relaxin...",chrome.exe,0,2024-09-07 13:55:10,0 days 00:00:08
5,6,09/07/2024,13:55:18,peaceful solitude - YouTube - Google Chrome,chrome.exe,0,2024-09-07 13:55:18,0 days 00:00:05
6,7,09/07/2024,13:55:23,focus.py - TimeLens - Visual Studio Code,Code.exe,0,2024-09-07 13:55:23,0 days 00:00:01
7,8,09/07/2024,13:55:24,Session End,None,1,2024-09-07 13:55:24,0 days 00:12:23
8,9,09/07/2024,14:07:47,● processing.ipynb - TimeLens - Visual Studio ...,Code.exe,0,2024-09-07 14:07:47,0 days 00:00:07
9,10,09/07/2024,14:07:54,Two Steps Ahead - YouTube - Google Chrome,chrome.exe,0,2024-09-07 14:07:54,0 days 00:00:04


In [42]:
#Overarching Insight section

#1. Context Switching
#How to get it? Get the time that we usually calculate for the time in focus_logs. Then, sum it up until we go to a different program, 
# then keep repeating and average it out. "Average Time Spent on a Task"
df['group'] = (df['session_end'].cumsum())
df

,id,date,time,focused,program,session_end,group
0,1,09/07/2024,13:54:43,focus.py - TimeLens - Visual Studio Code,Code.exe,0,0
1,2,09/07/2024,13:54:45,Efficient Time Calculation - Google Chrome,chrome.exe,0,0
2,3,09/07/2024,13:54:49,"Soft Jazz Music for Study, Work, Focus☕Relaxin...",chrome.exe,0,0
3,4,09/07/2024,13:55:09,Efficient Time Calculation - Google Chrome,chrome.exe,0,0
4,5,09/07/2024,13:55:10,"Soft Jazz Music for Study, Work, Focus☕Relaxin...",chrome.exe,0,0
...,...,...,...,...,...,...,...
69,70,09/07/2024,15:10:32,Group by: split-apply-combine — pandas 2.2.2 d...,chrome.exe,0,3
70,71,09/07/2024,15:10:41,pandas from one entry to another entry group -...,chrome.exe,0,3
71,72,09/07/2024,15:10:44,ChatGPT - Google Chrome,chrome.exe,0,3
72,73,09/07/2024,15:15:31,#amazon-feast | The Jade Palace - Discord,Discord.exe,0,3


In [8]:
# #First DF

# # At the beginning of all of this, in the raw csv, add a entry signifying the Tracking Ended. This is so we don't have
# # crazy gaps between ending and starting the next session. Add an ignorance for it.
# from datetime import datetime
# # Now by entry, I want to get a total time/add up datetimes. Join "Date" and "Time" then just sum up?
# df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
# #Makes a column w full datetime object

# # Sort by 'DateTime' to ensure correct time difference calculation
# df = df.sort_values(by=['datetime'])

# # Current datetime - next datetime
# df['timespent'] = df['datetime'].shift(-1) - df['datetime']
# # For the last row, set a default time (e.g., 5 seconds), as there's no next row
# df.iloc[-1, df.columns.get_loc('timespent')] = pd.Timedelta(seconds=5)

# # Group by 'Program' and sum the total time spent for each program
# total_time_per_program = df.groupby('program')['timespent'].sum()
# print(total_time_per_program.sort_values(ascending=False))


program
chrome.exe     0 days 00:00:13
Code.exe       0 days 00:00:08
explorer.exe   0 days 00:00:02
Name: timespent, dtype: timedelta64[ns]


In [10]:
# #Task: Storing insights into the database
# insights_data = {
#     'total_time': total_time_per_program
# }
# cursor = connection.cursor()

# # Insert or update insights for each program
# for program, total_time in total_time_per_program.items():

#     # Insert or update the row for the program
#     cursor.execute('''
#         INSERT INTO program_insights (program, total_time)
#         VALUES (?, ?)
#         ON CONFLICT(program) DO UPDATE SET 
#         total_time = excluded.total_time,
#         last_updated = CURRENT_TIMESTAMP
#     ''', (program, str(total_time)))
